In [2]:
%load_ext autoreload
%autoreload 2

In [17]:
from pathlib import Path
import json
from IPython.display import Latex, display
from openai import OpenAI
import numpy as np
from latex2sympy2 import latex2sympy
from tqdm.auto import tqdm
from datetime import datetime

from utils.string_utils import *
from utils.file_utils import *
from utils.generation_utils import *

In [4]:
client = OpenAI()

## Load Data


In [5]:
train_dir = Path("train")
train_files = np.array(list(train_dir.glob("*/*.json")))

In [6]:
train_files_by_category = {}

train_subdirs = list(train_dir.glob("*"))
print("Category Filecount:\n##########")
for train_subcategory in train_subdirs:
    train_files_by_category[train_subcategory.name] = np.array(
        list(train_subcategory.glob("*.json"))
    )
    print(
        f"{train_subcategory.name}: {len(train_files_by_category[train_subcategory.name])}"
    )

Category Filecount:
##########
counting_and_probability: 771
intermediate_algebra: 1295
counting_and_probability_sample: 430
number_theory: 869
precalculus: 746
prealgebra: 1205
geometry: 870
algebra: 1744


## Zero-Shot Generation


In [6]:
def generate(question_file):
    question = get_question_from_file(question_file)
    question["solution"] = question["solution"].replace("\\!", "")

    answer = generate_answer(client, question)
    matching = verify_answer(answer, question)
    return {
        "file_path": question_file.as_posix(),
        "question": question,
        "answer": answer,
        "final_answer": extract_answer(answer),
        "final_ground_truth": extract_answer(question["solution"]),
        "matching": matching,
    }

In [11]:
def generate_for_category(category, output_file):
    Path(f"generations/{category}").mkdir(parents=True, exist_ok=True)
    output_file = Path(f"generations/{category}/{output_file}.json")
    train_files = train_files_by_category[category]

    generations = {}
    if output_file.exists():
        with open(output_file) as f:
            generations = json.load(f)

    failures = []
    for file in tqdm(train_files):
        if file.as_posix() in generations:
            continue

        try:
            generations[file.as_posix()] = generate(file)
            with open(output_file, "w") as f:
                json.dump(generations, f)
        except:
            print(f"File {file} failed to generate")
            failures.append(file)

    print("done")
    return failures

In [12]:
curr_time = datetime.now().strftime("%Y-%m-%d")
generate_for_category("algebra", f"{curr_time}_run1")

  0%|          | 0/1744 [00:00<?, ?it/s]

File train/algebra/2075.json failed to generate
File train/algebra/1120.json failed to generate
File train/algebra/1931.json failed to generate
File train/algebra/768.json failed to generate
File train/algebra/1161.json failed to generate
File train/algebra/695.json failed to generate
File train/algebra/380.json failed to generate
File train/algebra/601.json failed to generate
File train/algebra/1771.json failed to generate
File train/algebra/2761.json failed to generate
File train/algebra/1337.json failed to generate
File train/algebra/2632.json failed to generate
File train/algebra/2645.json failed to generate
File train/algebra/2757.json failed to generate
File train/algebra/637.json failed to generate
File train/algebra/1497.json failed to generate
File train/algebra/1301.json failed to generate
File train/algebra/1751.json failed to generate
File train/algebra/1244.json failed to generate
File train/algebra/2346.json failed to generate
File train/algebra/363.json failed to generat

### Zero-Shot Troubleshooting


In [136]:
q = get_question_from_file(Path("train/counting_and_probability_sample/5122.json"))
a = generate_answer(q)

In [137]:
q

{'problem': 'Six distinct integers are picked at random from $\\{1,2,3,\\ldots,10\\}$. What is the probability that, among those selected, the second smallest is $3$?\n$\\textbf{(A)}\\ \\frac{1}{60}\\qquad \\textbf{(B)}\\ \\frac{1}{6}\\qquad \\textbf{(C)}\\ \\frac{1}{3}\\qquad \\textbf{(D)}\\ \\frac{1}{2}\\qquad \\textbf{(E)}\\ \\text{none of these}$\n',
 'level': 'Level 5',
 'type': 'Counting & Probability',
 'solution': 'The total number of ways to choose 6 numbers is ${10\\choose 6} = 210$.\nAssume $3$ is the second-lowest number. There are $5$ numbers left to choose, $4$ of which must be greater than $3$, and $1$ of which must be less than $3$. This is equivalent to choosing $4$ numbers from the $7$ numbers larger than $3$, and $1$ number from the $2$ numbers less than $3$.\\[{7\\choose 4} {2\\choose 1}= 35\\times2\\].\nThus, $\\frac{35\\times2}{210} = \\boxed{\\frac{1}{3}}$.'}

In [138]:
extract_answer(q["solution"]), extract_answer(a)

('\\frac{1}{3}', '\\frac{3}{5}')

In [139]:
latex2sympy("\\frac{1}{3}")

1/3

## Parrelized Generation


In [18]:
questions = get_questions_from_files(train_files[:5])
questions = preprocess_questions(questions)
messages = get_prompt_template_from_questions(questions)

In [20]:
data_json = []
for idx, message in enumerate(messages):
    data_json.append(
        {"model": "gpt-3.5-turbo", "messages": message, "metadata": {"row_id": idx}}
    )

In [22]:
with open('./test.jsonl', 'w') as out:
    for ddict in data_json:
        jout = json.dumps(ddict) + '\n'
        out.write(jout)